In [ ]:
#all_no_test
import os
import time
import json
from collections import defaultdict
from functools import reduce
import pandas as pd
import numpy as np
import lightgbm as lgb
from lightgbm.callback import record_evaluation
from opt_utils import *

os.system('pip install neptune-client')
os.system('pip install neptune-lightgbm')
import neptune.new as neptune
from neptune.new.integrations.lightgbm import create_booster_summary
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
NEPTUNE_API_TOKEN = user_secrets.get_secret("NEPTUNE_API_TOKEN")

cfg = {
    "script_name": 'opt_train_v2_nep_op_85_enc',
    "path_features": '../input/generate-train-features-script-p10/p10_train.pkl', # Used in train mode
    "path_models": '',
    "path_data_raw": '../input/optiver-realized-volatility-prediction/',
    "neptune_project": 'chrisrichardmiles/optiver',
    "neptune_description": 'p10 features second run with enc. Notice but and will have to rerun with feature generation and training',
    "encode_time_cols":  [
        'real_vol_mean_decay_0.99_-1', 'real_vol_mean_decay_0.95_-1', 
        'real_vol_mean_decay_0.9_-1', 'real_vol_mean_decay_0.85_-1', 
        'ask_price1_real_vol', 'bid_price1_real_vol', 'spread_mean', 
        'size_momentum', 'size_real_vol', 'abs_log_return_max_sub_min', 
        'sum_bid_ask_real_vol', 'log_return_mean_decay', 'wap_last-first'
                        ],
    "encode_stock_cols": [],
    "encode_funcs": ['mean', 'std', 'max', 'min'], 
    "shake": False, 
    "shake_std": .5, 
    "prefix": '',
    "rerun": True,
    "drop_cols": ['row_id', 'stock_id', 'time_id', 'target'] + ['abs_log_return_std_450', 'wap_last_150', 'seconds_in_bucket_count_unique_350', 'wap_mean_decay_150', 'price_wap_diff_mean_decay_flip_450', 'is_pos_return_max_sub_min_150', 'is_pos_return_mean_decay_150', 'wap_mean_350', 'abs_log_return_mean_decay_350', 'is_neg_return_mean_decay_0', 'log_return_max_sub_min_450', 'size_amin_350', 'abs_log_return_mean_decay_flip_350', 'abs_log_return_sum_350', 'is_neg_return_sum_450', 'is_neg_return_std_450', 'abs_log_return_std_150', 'log_return_count_150', 'abs_log_return_mean_decay_flip_450', 'size_sum_150', 'wap_mean_decay_350', 'is_pos_return_sum_0', 'size_sum_350', 'is_pos_return_sum_350', 'is_neg_return_max_sub_min_450', 'seconds_in_bucket_count_unique_150', 'is_pos_return_max_sub_min_450', 'price_wap_diff_amax_0', 'wap_last_350', 'wap_mean_decay_flip_150', 'wap_momentum_350', 'wap_mean_150', 'wap_amin_350', 'is_pos_return_std_350', 'size_amax_350', 'log_return_realized_volatility_150', 'wap_mean_450', 'log_return_std_150', 'order_count_amax_150', 'log_return_mean_decay_350', 'log_return_count_0', 'abs_log_return_sum_150', 'wap_amin_450', 'is_neg_return_sum_350', 'wap_amax_450', 'log_return_realized_volatility_350', 'is_pos_return_max_sub_min_350', 'wap_first_350', 'wap_first_450', 'sum_ask_sub_sum_bid_scaled_mean_decay_450', 'log_return_count_450', 'wap_last_450', 'log_return_std_450', 'size_amax_450', 'is_pos_return_mean_decay_350', 'is_neg_return_mean_decay_350', 'real_vol_mean_decay_0.45_-1', 'real_vol_mean_decay_0.85_1', 'sum_ask_sub_sum_bid_vol_log_return_150', 'sum_ask_sub_sum_bid_scaled_mean_350', 'log_return_std_350', 'log_return_realized_volatility_450', 'wap_mean_decay_450', 'is_pos_return_mean_decay_0', 'price_wap_diff_momentum_150', 'size_amin_150', 'wap_mean_decay_0', 'is_neg_return_mean_decay_flip_450', 'is_pos_return_sum_150', 'is_neg_return_std_350', 'log_return_std_0', 'size_amin_0', 'log_return_count_350', 'abs_log_return_mean_decay_150', 'real_vol_mean_decay_0.65_1', 'is_neg_return_mean_decay_150', 'wap_mean_decay_flip_450', 'is_neg_return_sum_150', 'wap_amax_0', 'wap_amax_350', 'is_neg_return_sum_0', 'ask_size1_vol_log_return_450', 'sum_ask_sub_sum_bid_vol_log_return_350', 'is_pos_return_std_450', 'price_vol_log_return_450', 'seconds_in_bucket_count_unique_0', 'wap_mean_decay_flip_350', 'price_wap_diff_momentum_350', 'wap_momentum_450', 'wap_mean_0', 'is_pos_return_max_sub_min_0', 'real_vol_mean_decay_0.55_-1', 'sum_ask_sub_sum_bid_scaled_mean_450', 'real_vol_mean_decay_0.9_1', 'log_return_mean_decay_flip_450', 'sum_ask_sub_sum_bid_scaled_mean_decay_0', 'wap_amin_0', 'is_neg_return_max_sub_min_150', 'log_return_mean_decay_flip_350', 'sum_ask_vol_log_return_350', 'abs_log_return_mean_decay_450', 'wap_amax_150', 'log_return_realized_volatility_0', 'is_pos_return_mean_decay_flip_450', 'is_pos_return_std_150', 'is_pos_return_mean_decay_450', 'price_wap_diff_mean_decay_0', 'sum_ask_sub_sum_bid_scaled_mean_decay_150', 'is_neg_return_mean_decay_450', 'price_wap_diff_mean_decay_350', 'wap_amin_150', 'is_neg_return_max_sub_min_0', 'is_pos_return_sum_450', 'is_neg_return_max_sub_min_350', 'sum_ask_sub_sum_bid_scaled_mean_decay_350', 'dummy1', 'seconds_in_bucket_count_unique_450', 'abs_log_return_mean_decay_0', 'price_wap_diff_mean_decay_150', 'abs_log_return_sum_450', 'size_sum_450', 
                         'is_neg_return_std_150'], 
    "neptune_run_name": '',
    "lgb_params": {
        # https://lightgbm.readthedocs.io/en/latest/index.html
        "boosting_type": "gbdt",
        "objective": "rmse",
        "learning_rate": 0.05,
        "num_leaves": 255,
        "min_data_in_leaf": 255,
        "feature_fraction": 0.8,
        "bagging_fraction": .5, # Select bagging_fraction of rows every bagging_freq of iterations.
        "bagging_freq": 1,      # This speeds up training and underfits. Need both set to do anything.
        "n_estimators": 5000,
        "early_stopping_rounds": 50,
        "n_jobs": -1,
        "seed": 42,
        "verbose": -1, 
    },
}
with open('cfg.json', 'w') as f: 
    json.dump(cfg, f)

In [ ]:
def rdf(train): 
    train['spe'] = ((train['target'] - train['pred']) / train['target']) ** 2
    return np.sqrt((train['spe'].sum()) / train.shape[0])

In [ ]:
with open('../input/nepop76/cfg.json') as f: 
    cfg = json.load(f)

In [ ]:
stocks = defaultdict(dict)

In [ ]:
all_train = pd.read_pickle('../input/nepop76/enc_p8_train.pkl')
cols = [c for c in all_train.columns if not c.startswith('stock_id_')]
all_train = all_train[cols]
# all_train.columns.tolist()
all_preds = all_train[['stock_id', 'target']]
all_preds['pred'] = 0

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,stock_id,target,pred
0,0,0.004136,0
1,0,0.001445,0
2,0,0.002168,0
3,0,0.002195,0
4,0,0.001747,0
...,...,...,...
428927,126,0.003461,0
428928,126,0.003113,0
428929,126,0.004070,0
428930,126,0.003357,0


In [ ]:
old_preds = all_train[['stock_id', 'target']]
old_preds['pred'] = np.load('../input/nepop76/oof_predictions.npy')

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
sdf = pd.DataFrame(columns=['old', 'new'])

In [ ]:
def main(stock_id): 
#     train = pd.read_pickle(cfg['path_features'])
#     train = encode_cols(train, cfg["encode_time_cols"], funcs=cfg['encode_funcs'], on='time_id')
    
    # Saving encoded stock columns
#     feat_file = 'enc_' + os.path.split(cfg['path_features'])[1]
#     tmp = encode_cols(train, cfg["encode_stock_cols"], funcs=cfg['encode_funcs'], on='stock_id')
#     tmp.to_pickle(os.path.join(cfg['path_models'], feat_file))
     
    train = all_train[all_train.stock_id == stock_id]
    drop_cols = [c for c in cfg['drop_cols'] if c in train.columns and c != 'stock_id' and c not in cfg['encode_stock_cols']]
    x = train.drop(drop_cols, axis = 1)
    y = train['target']
    
    oof_predictions = np.zeros(x.shape[0]) # Create out of folds array
    scores = [] # Keep track of scores for each fold and all oof at the end
    best_iterations = []
    training_best_scores = []
    valid_best_scores = [] # Same as scores in this script, but would be different with nested cv
    best_score_diffs = []
    dict_eval_logs = [] # For experimentation tracking
    booster_summaries = [] # For experimentation tracking
    dumb_features = []
    top_features = []
    
    for fold in range(5):
        trn_ind = x.fold != fold
        val_ind = x.fold == fold
        
        print(f'Training fold {fold}')
        x_train, x_val = x[trn_ind].drop('fold', axis=1), x[val_ind].drop('fold', axis=1)
        y_train, y_val = y[trn_ind], y[val_ind]
        
        x_train = encode_cols(x_train, 
                              cfg['encode_stock_cols'], 
                              funcs=cfg['encode_funcs'], 
                              shake=cfg['shake'], 
                              shake_std=cfg['shake_std']).drop('stock_id', axis=1)
        n_train_cols = x_train.shape[1]
        
        x_val = encode_cols(x_val, 
                            cfg['encode_stock_cols'], 
                            funcs=cfg['encode_funcs']).drop('stock_id', axis=1)
        
        train_weights = 1 / np.square(y_train) # Root mean squared percentage error weights
        val_weights = 1 / np.square(y_val)
        train_dataset = lgb.Dataset(x_train, y_train, weight=train_weights)
        val_dataset = lgb.Dataset(x_val, y_val, weight=val_weights, reference=train_dataset)
        
        dict_eval_log = {}
        model = lgb.train(params = cfg['lgb_params'], 
                          train_set = train_dataset, 
                          valid_sets = [val_dataset, train_dataset], 
                          valid_names = ['valid', 'train'], 
                          feval = feval_rmspe,
                          callbacks=[record_evaluation(dict_eval_log)],
                          verbose_eval = 50)
        
#         model.save_model(os.path.join(cfg['path_models'], f'{cfg["prefix"]}lgb_fold_{fold}.txt'))
        y_pred = model.predict(x_val)
        oof_predictions[val_ind] = y_pred
        scores.append(round(rmspe(y_val, y_pred), 3))
        
#         dumb_features.append(get_dumb_features(model))
#         top_features.append(get_top_features(model))
        
#         booster_summary = create_booster_summary(
#             booster=model,
#             log_importances=True,
#             max_num_features=25,
#             log_trees_as_dataframe=False, 
#             log_pickled_booster=True, 
#             y_true=y_val, 
#             y_pred=y_pred, 
#         )
#         train_score = model.best_score['train']['RMSPE']
#         valid_score = model.best_score['valid']['RMSPE']
#         best_iterations.append(model.best_iteration)
#         training_best_scores.append(round(train_score, 3))
#         valid_best_scores.append(round(valid_score, 3))
#         best_score_diffs.append(round(valid_score - train_score, 3))
        
#         booster_summaries.append(booster_summary)
#         dict_eval_logs.append(dict_eval_log)
#         del booster_summary, dict_eval_log
    
    
    rmspe_score = round(rmspe(y, oof_predictions), 3)
    old = rdf(old_preds[old_preds.stock_id == stock_id])
    sc = [old, rmspe_score]
    sdf.loc[stock_id] = sc
    all_preds.loc[all_preds.stock_id == stock_id, 'pred'] = oof_predictions
    print(f'Our out of folds RMSPE is {rmspe_score}, compared to {old}, giving gain {rmspe_score - old}')
    print(f'Our cv fold scores are {scores}')
#     np.save('oof_predictions', oof_predictions)
    
#     run = neptune.init(
#             project=cfg['neptune_project'],
#             api_token=NEPTUNE_API_TOKEN,
#             name=cfg['neptune_run_name'],    
#             description=cfg['neptune_description'],
#             tags=[cfg['path_features'], cfg['prefix']],
#             source_files=['cfg.json'],
#     )
#     run = stocks[stock_id]
# #     run['feat_id'] = feat_file
#     run['old_score'] = 
#     run['cfg'] = cfg
#     run['RMSPE'] = rmspe_score
#     run['RMSPE_oof_scores'] = scores
#     run['RMSPE_cv_std'] = np.std(scores)
    
#     run['best_iterations'] = best_iterations
#     best_iterations_mean = int(np.mean(best_iterations))
#     run['best_iterations_mean'] = best_iterations_mean
#     run['training_best_scores'] = training_best_scores
#     run['valid_best_scores'] = valid_best_scores
#     run['best_score_diffs'] = best_score_diffs
#     run['best_score_diffs_mean'] = round(np.mean(best_score_diffs), 3)
#     run['dumb_features'] = list(reduce(lambda a, b: set(a).intersection(set(b)), dumb_features))
#     run['top_features'] = list(reduce(lambda a, b: set(a).intersection(set(b)), top_features))
    
#     run[f'fold_{fold}'] = booster_summaries[fold]
#     run[f'dumb_features_{fold}'] = list(dumb_features[fold])
#     run[f'top_features_{fold}'] = list(top_features[fold])
    
    # Logs for each folds model
#     for fold in range(5):
#         run[f'lgbm_summaries/fold_{fold}'] = booster_summaries[fold]
#         run[f'lgbm_summaries/dumb_features_{fold}'] = list(dumb_features[fold])
#         run[f'lgbm_summaries/top_features_{fold}'] = list(top_features[fold])
#         dict_eval_log = dict_eval_logs[fold]
#         for valid_set, odict in dict_eval_log.items():
#             for metric, log in odict.items():
#                 for val in log:
#                     run[f'eval_logs/{fold}_{valid_set}_{metric}'].log(val)
#     run.stop()
    
#     if cfg['rerun']: 
#         print(f'retraining model with all data for {best_iterations} iterations')
#         params = cfg['lgb_params'].copy()
#         params['early_stopping_rounds'] = 0 # No valid set to stop with
        
#         x_train = x.drop(['fold'], axis=1)
#         x_train = encode_cols(x_train, 
#                               cfg['encode_stock_cols'], 
#                               funcs=cfg['encode_funcs'], 
#                               shake=cfg['shake'], 
#                               shake_std=cfg['shake_std']).drop('stock_id', axis=1)
#         y_train = y
        
#         assert(n_train_cols == x_train.shape[1])
        
#         train_weights = 1 / np.square(y_train) # Root mean squared percentage error weights
#         train_dataset = lgb.Dataset(x_train, y_train, weight=train_weights)
        
#         for fold, best_iter in enumerate(best_iterations): 
#             params['n_estimators'] = int(best_iter) # lgbm needs int here
#             model = lgb.train(params = params, 
#                               train_set = train_dataset)
#             model.save_model(os.path.join(cfg['path_models'], f'{cfg["prefix"]}rerun_lgb_{fold}.txt'))
    
# if __name__ == '__main__': 
#     main()

In [ ]:
main(31)

Training fold 0
Training until validation scores don't improve for 50 rounds


/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))
/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


[50]	train's rmse: 0.000613071	train's RMSPE: 0.440295	valid's rmse: 0.000770026	valid's RMSPE: 0.444244
[100]	train's rmse: 0.000542566	train's RMSPE: 0.38966	valid's rmse: 0.000754667	valid's RMSPE: 0.435383
Early stopping, best iteration is:
[88]	train's rmse: 0.00055269	train's RMSPE: 0.396931	valid's rmse: 0.000751319	valid's RMSPE: 0.433451
Training fold 1


/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))
/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 50 rounds
[50]	train's rmse: 0.000601588	train's RMSPE: 0.419415	valid's rmse: 0.00116532	valid's RMSPE: 0.783223
Early stopping, best iteration is:
[16]	train's rmse: 0.000776851	train's RMSPE: 0.541605	valid's rmse: 0.0010316	valid's RMSPE: 0.693344
Training fold 2


...

/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))
/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 50 rounds
[50]	train's rmse: 0.000668864	train's RMSPE: 0.440228	valid's rmse: 0.000588563	valid's RMSPE: 0.479311
[100]	train's rmse: 0.000600592	train's RMSPE: 0.395293	valid's rmse: 0.000617043	valid's RMSPE: 0.502504
Early stopping, best iteration is:
[57]	train's rmse: 0.000652315	train's RMSPE: 0.429336	valid's rmse: 0.000576923	valid's RMSPE: 0.469831
Training fold 4


/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))
/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 50 rounds
[50]	train's rmse: 0.000645764	train's RMSPE: 0.448808	valid's rmse: 0.000658246	valid's RMSPE: 0.448308
[100]	train's rmse: 0.000564414	train's RMSPE: 0.39227	valid's rmse: 0.000672598	valid's RMSPE: 0.458083
Early stopping, best iteration is:
[57]	train's rmse: 0.0006261	train's RMSPE: 0.435142	valid's rmse: 0.000649591	valid's RMSPE: 0.442413
Our out of folds RMSPE is 0.508, compared to 0.5555796745804047, giving gain -0.047579674580404685
Our cv fold scores are [0.433, 0.693, 0.452, 0.47, 0.442]


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [ ]:
for stock_id in all_preds.stock_id.unique(): 
    main(stock_id)

Training fold 0
Training until validation scores don't improve for 50 rounds


/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))
/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


[50]	train's rmse: 0.000708746	train's RMSPE: 0.275569	valid's rmse: 0.00077724	valid's RMSPE: 0.296384
[100]	train's rmse: 0.000678066	train's RMSPE: 0.26364	valid's rmse: 0.000767171	valid's RMSPE: 0.292545
[150]	train's rmse: 0.000661326	train's RMSPE: 0.257131	valid's rmse: 0.000765755	valid's RMSPE: 0.292005
[200]	train's rmse: 0.00064754	train's RMSPE: 0.251771	valid's rmse: 0.000763377	valid's RMSPE: 0.291098
[250]	train's rmse: 0.00063573	train's RMSPE: 0.247179	valid's rmse: 0.000760109	valid's RMSPE: 0.289852
[300]	train's rmse: 0.000625967	train's RMSPE: 0.243383	valid's rmse: 0.000758097	valid's RMSPE: 0.289085
[350]	train's rmse: 0.000615999	train's RMSPE: 0.239507	valid's rmse: 0.000757436	valid's RMSPE: 0.288833
[400]	train's rmse: 0.00060813	train's RMSPE: 0.236448	valid's rmse: 0.000755773	valid's RMSPE: 0.288198
[450]	train's rmse: 0.000601009	train's RMSPE: 0.233679	valid's rmse: 0.000753786	valid's RMSPE: 0.287441
[500]	train's rmse: 0.000594125	train's RMSPE: 0.231

/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))
/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 50 rounds
[50]	train's rmse: 0.000717123	train's RMSPE: 0.277548	valid's rmse: 0.000735287	valid's RMSPE: 0.285668
[100]	train's rmse: 0.000687241	train's RMSPE: 0.265983	valid's rmse: 0.000723043	valid's RMSPE: 0.280911
[150]	train's rmse: 0.000670054	train's RMSPE: 0.259331	valid's rmse: 0.000721038	valid's RMSPE: 0.280132
[200]	train's rmse: 0.00065537	train's RMSPE: 0.253647	valid's rmse: 0.000720285	valid's RMSPE: 0.27984
[250]	train's rmse: 0.000643291	train's RMSPE: 0.248973	valid's rmse: 0.000723496	valid's RMSPE: 0.281087
Early stopping, best iteration is:
[204]	train's rmse: 0.000653848	train's RMSPE: 0.253058	valid's rmse: 0.000719287	valid's RMSPE: 0.279452
Training fold 2


...

/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))
/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 50 rounds
[50]	train's rmse: 0.000840108	train's RMSPE: 0.232462	valid's rmse: 0.000889966	valid's RMSPE: 0.247916
[100]	train's rmse: 0.000800433	train's RMSPE: 0.221483	valid's rmse: 0.000867968	valid's RMSPE: 0.241788
[150]	train's rmse: 0.000780858	train's RMSPE: 0.216067	valid's rmse: 0.000867771	valid's RMSPE: 0.241733
[200]	train's rmse: 0.000765963	train's RMSPE: 0.211945	valid's rmse: 0.000865082	valid's RMSPE: 0.240984
Early stopping, best iteration is:
[188]	train's rmse: 0.00076895	train's RMSPE: 0.212772	valid's rmse: 0.000863735	valid's RMSPE: 0.240609
Training fold 4


/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))
/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 50 rounds
[50]	train's rmse: 0.000850583	train's RMSPE: 0.236059	valid's rmse: 0.000853338	valid's RMSPE: 0.234907
[100]	train's rmse: 0.000811834	train's RMSPE: 0.225305	valid's rmse: 0.000829466	valid's RMSPE: 0.228335
[150]	train's rmse: 0.00079083	train's RMSPE: 0.219476	valid's rmse: 0.000832288	valid's RMSPE: 0.229112
Early stopping, best iteration is:
[100]	train's rmse: 0.000811834	train's RMSPE: 0.225305	valid's rmse: 0.000829466	valid's RMSPE: 0.228335
Our out of folds RMSPE is 0.233, compared to 0.2129300098765868, giving gain 0.020069990123413206
Our cv fold scores are [0.233, 0.235, 0.227, 0.241, 0.228]


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [ ]:
rdf(all_preds)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


0.2449659904286853

In [ ]:
sdf['diff'] = sdf['new'] - sdf['old']
sdf[sdf['diff'] < 0]

,old,new,diff
31,0.55558,0.506,-0.04958


In [ ]:
rdf(old_preds[old_preds.stock_id != 31])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


0.21287723151184676

In [ ]:
rdf(old_preds)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


0.21833054495346182

In [ ]:
tmp = old_preds.copy()
tmp.loc[tmp.stock_id == 31, 'pred'] = all_preds.loc[all_preds.stock_id == 31, 'pred']

In [ ]:
sdf.loc[[31, 37]]

,old,new,diff
31,0.555580,0.506,-0.049580
37,0.299569,0.311,0.011431


In [ ]:
sdf.sort_values('diff')

,old,new,diff
31,0.555580,0.506,-0.049580
81,0.257210,0.260,0.002790
18,0.288923,0.295,0.006077
6,0.203179,0.214,0.010821
37,0.299569,0.311,0.011431
...,...,...,...
51,0.175530,0.221,0.045470
68,0.207333,0.255,0.047667
39,0.196862,0.245,0.048138
34,0.186580,0.235,0.048420


In [ ]:
sdf.loc[80]

old     0.25266
new     0.27000
diff    0.01734
Name: 80, dtype: float64